# Limpieza y transformación de datos

## Preparando el ambiente

### Cargando modulos

In [1]:
# Cargando módulos
# ---------------------------------------------------------------------------------------
# Configuración warnings
# Importando modulo warnings
import warnings

# Ignorar warnings
warnings.filterwarnings("ignore")

# Importando importOrInstall function desde el modulo lib.functChecModule
from lib.functChecModule import importOrInstall

# Use the importOrInstall function para validar si el modulo 'yaml' module se encuentra instalado, y si es necesario lo instala
importOrInstall("yaml")

# Use the importOrInstall function para validar si el modulo 'skimpy' module se encuentra instalado, y si es necesario lo instala
importOrInstall('skimpy')

In [2]:
# Manipulando datos
# Importando varios módulos desde pandas
from pandas import DataFrame, read_csv, set_option, option_context, to_datetime, Series, Timedelta, merge

# Importando varios módulos desde numpy
from numpy import nan, where

# Importando la clase SafeLoader desde el modulo yaml.loader
from yaml.loader import SafeLoader

# Importando los módulos json y yaml
import json
import yaml

# Importando el módulo statistics
import statistics

# Importando la función clean_columns desde el módulo skimpy
from skimpy import clean_columns

# Importando varios módulos desde datetime
from datetime import date, datetime, timedelta

# Importando el módulo missingno y asignando el alias msno
import missingno as msno

# Importando el módulo pickle
import pickle

# Importando los módulos os y glob
import os
import glob

# Importando varios módulos desde psycopg2
# import psycopg2
# from psycopg2 import connect, extensions, sql

# Importando la función create_engine desde sqlalchemy
# from sqlalchemy import create_engine

# Importando funciones custom custom desde el módulo lib.functDescriptiveAnalysis 
from lib.functDescriptiveAnalysis import inspectDataFrameDA, countNaNDA

# Importando funciones custom custom desde el módulo lib.functCleanData
from lib.functCleanData import convert_float, whitespaceRemover

### Eliminando archivos plk

In [3]:
# Obtiene archivos desde la carpeta de salida
# Este código en Python recorre un directorio en busca de archivos con extensión ".plk" y los elimina.
# Si no se encuentran archivos con esa extensión, se imprime un mensaje indicando que no se encontraron 
# archivos en el directorio especificado. La línea lstPlk = [f for f in os.listdir('data/output/') if f.endswith('.plk')] 
# utiliza una list comprehension para crear una lista de archivos que cumplan la condición de tener la extensión ".plk".
# La siguiente sección del código, if(len(lstPlk) > 0):, verifica si la lista creada anteriormente contiene algún elemento. 
# Si contiene al menos un elemento, se recorre la lista con un ciclo for y se elimina cada archivo con la función os.remove().
# Si se produce un error al intentar eliminar un archivo, se imprime un mensaje con el error y el nombre del archivo.
# Si no se encuentran archivos en el directorio, se imprime un mensaje indicando que no se encontraron archivos en el directorio especificado.

lstPlk = [f for f in os.listdir('data/output/') if f.endswith('.plk')]
if(len(lstPlk) > 0):
    for fName in lstPlk:
        try:
            os.remove('data/output/' + fName)
            print("Archivo %s removido exitosamente" % fName)
        except OSError as error:
            print(error)
            print("Archivo %s no pudo ser removido" % fName)
else:
    print("No se encontraron archivos en el directorio data/output/")

Archivo purchase_order.plk removido exitosamente
Archivo product_order.plk removido exitosamente
Archivo shopper.plk removido exitosamente
Archivo store.plk removido exitosamente
Archivo dataset_full_data.plk removido exitosamente
Archivo dataset_pre_modelacion.plk removido exitosamente


## Cargando dataset's

### Dataset: orden de compra (purchase_order)

In [4]:
# Preparando los datos
# ---------------------------------------------------------------------------------------
# Cargando datos
# Cargando datos de la orden de compra desde el archivo especificado
purchase_order = read_csv('data/input/1_Orden_compra.csv', sep = ';', encoding='utf-8')
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
    # Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(purchase_order.head(n = 5))

,order_id,lat,lng,dow,promised_time,actual_time,on_demand,picker_id,driver_id,store_branch_id,total_minutes
0,6abd5e28d01b7c733fe37485294235dc,1.822.114.109.606.680,-10.031.391.366.664.600,2,13:00:00,12:00:15,False,09bb5bfe76adbb474dddbdd111a51623,09bb5bfe76adbb474dddbdd111a51623,0dc6fb5924a8b917bc641666ed02663c,91.800.861
1,0d37a1eea001f01b40232ffd496ff663,18.177.712.202.580.300,-10.035.705.073.096.900,2,13:00:00,10:28:10,False,c4c722114a82fa21654e57e9636d68b0,c4c722114a82fa21654e57e9636d68b0,2bbf833e689d24b697f1f3529c3f3c4b,206.599.732
2,528428e9b9eb5d3bd0dbdd8da692b087,1.826.683.205.404.410,-10.031.755.280.890.600,2,11:00:00,10:40:50,False,e15c7a67d0ea9a54dffb410bc1d8528b,e15c7a67d0ea9a54dffb410bc1d8528b,8d891dfc0e29d744c15c18adced74a68,93.458.001
3,d3f7be6d64eae7405b7bf024e04807a2,18.248.884.004.419.400,-1.003.226.215.187.040,2,10:00:00,9:41:47,False,d1f0bd4b237c1358f09db22076b940e7,d1f0bd4b237c1358f09db22076b940e7,0dc6fb5924a8b917bc641666ed02663c,124.778.761
4,e0ad6533b6cd9a382ec6c1d473c5036c,1.822.911.027.242.010,-10.032.267.241.783.800,2,11:00:00,10:07:53,False,06a71e8ab0c6db5ce22a4689fd76a3f4,06a71e8ab0c6db5ce22a4689fd76a3f4,afd0a7b8d546fcc697bb93a5934b65c3,130.321.208


In [5]:
# Otiene una lista de nombres de columnas de un DataFrame de pandas.
purchase_order.columns.to_list()

['order_id',
 'lat',
 'lng',
 'dow',
 'promised_time',
 'actual_time',
 'on_demand',
 'picker_id',
 'driver_id',
 'store_branch_id',
 'total_minutes']

#### Normalizando nombre de columnas

In [6]:
# Asigna al dataframe el resultado de llamar a la función clean_columns().
# Esta función tiene como objetivo limpiar los nombres de las columnas del DataFrame.
# Elimina los caracteres especiales y los espacios los reemplaza por un _
purchase_order = clean_columns(purchase_order, case = 'snake')
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
purchase_order.columns.to_list()

0 column names have been cleaned

['order_id',
 'lat',
 'lng',
 'dow',
 'promised_time',
 'actual_time',
 'on_demand',
 'picker_id',
 'driver_id',
 'store_branch_id',
 'total_minutes']

#### Eliminando valores perdidos

In [7]:
# Limpieza de datos
# ---------------------------------------------------------------------------------------
# Eliminando valores perdidos
purchase_order.dropna(inplace=True)
purchase_order = purchase_order.reset_index(drop=True)

#### Eliminando valores duplicados

In [8]:
# Eliminando valores duplicados
purchase_order = purchase_order.drop_duplicates()
purchase_order = purchase_order.reset_index(drop=True)

#### Normalizando columnas para eliminar puntos adicionales y ajustando largo de decimales

In [9]:
# Normalizando columnas de tipo objeto a float64
#Transformar la columna "total_minutes" de un DataFrame.
# La función DataFrame.map() aplica una función lambda a cada elemento de la columna seleccionada
# y devuelve una serie con los resultados.
# En este caso, la función lambda recibe como argumento x y llama a una función externa llamada convert_float() pasando x como argumento.
# Esta función tiene como objetivo convertir un string a un número de punto flotante.
# En ete caso elimina los puntos "sobrantes" despues del primer punto de izquiera a derecha
purchase_order['total_minutes'] = purchase_order['total_minutes'].map(lambda x: convert_float(x))

# eliminando . (puntos) despues del primer punto a la izquierda
# transformando las columnas "lat" y "lng" del DataFrame.
# La función DataFrame.map() aplica una función lambda a cada elemento de la columna seleccionada
# y devuelve una serie con los resultados.
# En este caso, la función lambda recibe como argumento x y reemplaza cada "." en el string con una cadena vacía
# (es decir, elimina los puntos decimales). Luego, 
# se extraen los primeros dos dígitos para la columna "lat" 
# y los primeros tres dígitos para la columna "lng",
# luego se concatenan con un nuevo punto decimal y los siguientes cinco dígitos del string original 
# (considerando la posición inicial dependiendo de la columna).
# Por último, se utiliza la función Series.astype() para cambiar el tipo de datos de las columnas "lat" y "lng" a float64.
# Esto se hace para que los valores de las columnas puedan ser tratados como números.

# purchase_order['lat'] = purchase_order['lat'].map(lambda x: x[0:x.find('.')] + '.' + x[x.find('.'):].replace('.', '')[0:5]).astype(float)
# purchase_order['lng'] = purchase_order['lng'].map(lambda x: x[0:x.find('.')] + '.' + x[x.find('.'):].replace('.', '')[0:5]).astype(float)
purchase_order['lat'] = purchase_order['lat'].map(lambda x: x.replace('.', '')[:2] + '.' + x.replace('.', '')[2:7]).astype('float64')
purchase_order['lng'] = purchase_order['lng'].map(lambda x: x.replace('.', '')[:3] + '.' + x.replace('.', '')[3:8]).astype('float64')

##### Inspeccion visual transformaciones

In [10]:
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(purchase_order.head(n = 5))

,order_id,lat,lng,dow,promised_time,actual_time,on_demand,picker_id,driver_id,store_branch_id,total_minutes
0,6abd5e28d01b7c733fe37485294235dc,18.22114,-10.03139,2,13:00:00,12:00:15,False,09bb5bfe76adbb474dddbdd111a51623,09bb5bfe76adbb474dddbdd111a51623,0dc6fb5924a8b917bc641666ed02663c,91.800861
1,0d37a1eea001f01b40232ffd496ff663,18.17771,-10.03570,2,13:00:00,10:28:10,False,c4c722114a82fa21654e57e9636d68b0,c4c722114a82fa21654e57e9636d68b0,2bbf833e689d24b697f1f3529c3f3c4b,206.599732
2,528428e9b9eb5d3bd0dbdd8da692b087,18.26683,-10.03175,2,11:00:00,10:40:50,False,e15c7a67d0ea9a54dffb410bc1d8528b,e15c7a67d0ea9a54dffb410bc1d8528b,8d891dfc0e29d744c15c18adced74a68,93.458001
3,d3f7be6d64eae7405b7bf024e04807a2,18.24888,-10.03226,2,10:00:00,9:41:47,False,d1f0bd4b237c1358f09db22076b940e7,d1f0bd4b237c1358f09db22076b940e7,0dc6fb5924a8b917bc641666ed02663c,124.778761
4,e0ad6533b6cd9a382ec6c1d473c5036c,18.22911,-10.03226,2,11:00:00,10:07:53,False,06a71e8ab0c6db5ce22a4689fd76a3f4,06a71e8ab0c6db5ce22a4689fd76a3f4,afd0a7b8d546fcc697bb93a5934b65c3,130.321208


#### Creando atributos calculados

In [11]:
# Transformando atributos de tipo datetime, time
#purchase_order['actual_reception_time'] = to_datetime(purchase_order['actual_time'].str.strip(), format = '%H:%M:%S')
#purchase_order['promised_delivery_time'] = to_datetime(purchase_order['promised_time'].str.strip(), format='%H:%M:%S')
purchase_order['actual_reception_time'] = purchase_order['actual_time'].apply(lambda x: to_datetime(x.strip(), format = '%H:%M:%S') + timedelta(1) if str(x)[:x.find(':')] == '0' else to_datetime(x.strip(), format = '%H:%M:%S'))
purchase_order['promised_delivery_time'] = purchase_order['promised_time'].apply(lambda x: to_datetime(x.strip(), format = '%H:%M:%S') + timedelta(1) if str(x)[:x.find(':')] == '0' else to_datetime(x.strip(), format = '%H:%M:%S'))
purchase_order['total_minutes_td'] = Series([Timedelta(minutes = row) for row in purchase_order['total_minutes']])
purchase_order['order_time'] = purchase_order['actual_reception_time'] - purchase_order['total_minutes_td']
purchase_order['model_1_dt_time'] = purchase_order['promised_delivery_time'] - purchase_order['order_time']
purchase_order['model_1_time'] = Series([row.total_seconds()/60 for row in purchase_order['model_1_dt_time']])
purchase_order = purchase_order[purchase_order['order_time'] < purchase_order['promised_delivery_time']]

##### Inspección visual transformaciones

In [12]:
with option_context("display.max_columns", None):
    display(purchase_order.head(n = 5))

,order_id,lat,lng,dow,promised_time,actual_time,on_demand,picker_id,driver_id,store_branch_id,total_minutes,actual_reception_time,promised_delivery_time,total_minutes_td,order_time,model_1_dt_time,model_1_time
0,6abd5e28d01b7c733fe37485294235dc,18.22114,-10.03139,2,13:00:00,12:00:15,False,09bb5bfe76adbb474dddbdd111a51623,09bb5bfe76adbb474dddbdd111a51623,0dc6fb5924a8b917bc641666ed02663c,91.800861,1900-01-01 12:00:15,1900-01-01 13:00:00,0 days 01:31:48.051660,1900-01-01 10:28:26.948340000,0 days 02:31:33.051660,151.550861
1,0d37a1eea001f01b40232ffd496ff663,18.17771,-10.03570,2,13:00:00,10:28:10,False,c4c722114a82fa21654e57e9636d68b0,c4c722114a82fa21654e57e9636d68b0,2bbf833e689d24b697f1f3529c3f3c4b,206.599732,1900-01-01 10:28:10,1900-01-01 13:00:00,0 days 03:26:35.983919999,1900-01-01 07:01:34.016080001,0 days 05:58:25.983919999,358.433065
2,528428e9b9eb5d3bd0dbdd8da692b087,18.26683,-10.03175,2,11:00:00,10:40:50,False,e15c7a67d0ea9a54dffb410bc1d8528b,e15c7a67d0ea9a54dffb410bc1d8528b,8d891dfc0e29d744c15c18adced74a68,93.458001,1900-01-01 10:40:50,1900-01-01 11:00:00,0 days 01:33:27.480060,1900-01-01 09:07:22.519940000,0 days 01:52:37.480060,112.624668
3,d3f7be6d64eae7405b7bf024e04807a2,18.24888,-10.03226,2,10:00:00,9:41:47,False,d1f0bd4b237c1358f09db22076b940e7,d1f0bd4b237c1358f09db22076b940e7,0dc6fb5924a8b917bc641666ed02663c,124.778761,1900-01-01 09:41:47,1900-01-01 10:00:00,0 days 02:04:46.725660,1900-01-01 07:37:00.274340000,0 days 02:22:59.725660,142.995428
4,e0ad6533b6cd9a382ec6c1d473c5036c,18.22911,-10.03226,2,11:00:00,10:07:53,False,06a71e8ab0c6db5ce22a4689fd76a3f4,06a71e8ab0c6db5ce22a4689fd76a3f4,afd0a7b8d546fcc697bb93a5934b65c3,130.321208,1900-01-01 10:07:53,1900-01-01 11:00:00,0 days 02:10:19.272480,1900-01-01 07:57:33.727520000,0 days 03:02:26.272480,182.437875


In [13]:
purchase_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6503 entries, 0 to 6542
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   order_id                6503 non-null   object         
 1   lat                     6503 non-null   float64        
 2   lng                     6503 non-null   float64        
 3   dow                     6503 non-null   int64          
 4   promised_time           6503 non-null   object         
 5   actual_time             6503 non-null   object         
 6   on_demand               6503 non-null   bool           
 7   picker_id               6503 non-null   object         
 8   driver_id               6503 non-null   object         
 9   store_branch_id         6503 non-null   object         
 10  total_minutes           6503 non-null   float64        
 11  actual_reception_time   6503 non-null   datetime64[ns] 
 12  promised_delivery_time  6503 non-n

#### Guardando dataset

In [14]:
#  La función pickle.dump() permite guardar el dataframe en un archivo en formato plk
pickle.dump(purchase_order, open('data/output/purchase_order.plk', 'wb'))

### Dataset: orden de productos (product_order)

In [15]:
# Cargando datos de la orden de productos desde el archivo especificado
product_order = read_csv('data/input/2_Orden_productos.csv', sep = ';', encoding='utf-8')
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(product_order.head(n = 5))

,order_id,product_id,quantity,quantity_found,buy_unit
0,bdfc44fb525986b7bab8c589fc47e853,ab9757d9892b0e83ab3a1c1af0c84c52,0.75,0.155,KG
1,bdfc44fb525986b7bab8c589fc47e853,f1f042251bf634d44f338fbaf13b16e4,0.3,0.400,KG
2,bdfc44fb525986b7bab8c589fc47e853,0630d47440965509b0b3f80fd1b3578f,1,1.000,UN
3,bdfc44fb525986b7bab8c589fc47e853,9dd69b1f300666d5573e3a9934f2e03e,2,2.000,UN
4,bdfc44fb525986b7bab8c589fc47e853,3dfcdbe308bde67370649dd875f2507b,1,1.000,UN


In [16]:
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
product_order.columns.to_list()

['order_id', 'product_id', 'quantity', 'quantity_found', 'buy_unit']

#### Normalizando nombre de columnas

In [17]:
# Asigna al dataframe el resultado de llamar a la función clean_columns().
# Esta función tiene como objetivo limpiar los nombres de las columnas del DataFrame.
# Elimina los caracteres especiales y los espacios los reemplaza por un _
product_order = clean_columns(product_order, case = 'snake')
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
product_order.columns.to_list()

0 column names have been cleaned

['order_id', 'product_id', 'quantity', 'quantity_found', 'buy_unit']

#### Imputando valores perdidos

In [18]:
# Dataset no tiene atributos valores perdidos

#### Imputando valores duplicados

In [19]:
# Eliminando valores duplicados
product_order = product_order.drop_duplicates()
product_order = product_order.reset_index(drop=True)

#### Normalizando columnas para eliminar puntos adicionales y ajustando largo de decimales

In [20]:
for i in product_order.index: 
    vQuantity = product_order['quantity'][i]
    vPedido = where(int(product_order['quantity_found'][i]) > 10, 2, 1)
    product_order['quantity'][i] = float(vQuantity.replace('.', '')[:vPedido] + '.' + vQuantity.replace('.', '')[vPedido:5] if vQuantity.count('.') > 1 else vQuantity)

##### Inspeccion visual transformaciones

In [21]:
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(product_order.head(n = 5))

,order_id,product_id,quantity,quantity_found,buy_unit
0,bdfc44fb525986b7bab8c589fc47e853,ab9757d9892b0e83ab3a1c1af0c84c52,0.75,0.155,KG
1,bdfc44fb525986b7bab8c589fc47e853,f1f042251bf634d44f338fbaf13b16e4,0.3,0.400,KG
2,bdfc44fb525986b7bab8c589fc47e853,0630d47440965509b0b3f80fd1b3578f,1.0,1.000,UN
3,bdfc44fb525986b7bab8c589fc47e853,9dd69b1f300666d5573e3a9934f2e03e,2.0,2.000,UN
4,bdfc44fb525986b7bab8c589fc47e853,3dfcdbe308bde67370649dd875f2507b,1.0,1.000,UN


In [22]:
product_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120873 entries, 0 to 120872
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   order_id        120873 non-null  object 
 1   product_id      120873 non-null  object 
 2   quantity        120873 non-null  object 
 3   quantity_found  120873 non-null  float64
 4   buy_unit        120873 non-null  object 
dtypes: float64(1), object(4)
memory usage: 4.6+ MB


#### Guardando dataset

In [23]:
#  La función pickle.dump() permite guardar el dataframe en un archivo en formato plk
pickle.dump(product_order, open('data/output/product_order.plk', 'wb'))

### Dataset: recolector (shopper)

In [24]:
# Cargando datos de recolector desde el archivo especificado
shopper = read_csv('data/input/3_Recolector.csv', sep = ';', encoding='utf-8')
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(shopper.head(n = 5))

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,087ab65ceca40dbef697f585dc159d5d,ADVANCED,0.8610,1.64,1.00,4.48
1,645973c41314d763b58dc2bceeeeffc9,INTERMEDIATE,0.8337,2.83,0.96,4.79
2,82810715fdda44962e7e422015ea1f32,ADVANCED,0.8681,1.27,0.88,4.84
3,416aa7dbd5a2f58d7bca96657e8faa1e,INTERMEDIATE,0.8784,2.11,1.00,NaN
4,27a3e845a4c3eb7ded4748f84140a843,ADVANCED,0.8466,1.50,1.00,4.84


In [25]:
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
shopper.columns.to_list()

['shopper_id',
 'seniority',
 'found_rate',
 'picking_speed',
 'accepted_rate',
 'rating']

#### Normalizando nombre de columnas

In [26]:
# Asigna al dataframe el resultado de llamar a la función clean_columns().
# Esta función tiene como objetivo limpiar los nombres de las columnas del DataFrame.
# Elimina los caracteres especiales y los espacios los reemplaza por un _
shopper = clean_columns(shopper, case = 'snake')
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
shopper.columns.to_list()

0 column names have been cleaned

['shopper_id',
 'seniority',
 'found_rate',
 'picking_speed',
 'accepted_rate',
 'rating']

#### Imputando valores perdidos

In [27]:
shopper["rating"] = shopper["rating"].replace(nan, shopper["rating"].mean()).round(2)
shopper["found_rate"] = shopper["found_rate"].replace(nan, shopper["found_rate"].mean()).round(2)
shopper["accepted_rate"] = shopper["accepted_rate"].replace(nan, shopper["accepted_rate"].mean()).round(2)

#### Normalizando columnas para eliminar puntos adicionales y ajustando largo de decimales

In [28]:
# Dataset no tiene atributos con esta condicion

#### Imputando valores duplicados

In [29]:
shopper = shopper.drop_duplicates()
shopper = shopper.reset_index(drop=True)

##### Inspección visual transformaciones

In [30]:
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(shopper.head(n = 5))

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,087ab65ceca40dbef697f585dc159d5d,ADVANCED,0.86,1.64,1.00,4.48
1,645973c41314d763b58dc2bceeeeffc9,INTERMEDIATE,0.83,2.83,0.96,4.79
2,82810715fdda44962e7e422015ea1f32,ADVANCED,0.87,1.27,0.88,4.84
3,416aa7dbd5a2f58d7bca96657e8faa1e,INTERMEDIATE,0.88,2.11,1.00,4.79
4,27a3e845a4c3eb7ded4748f84140a843,ADVANCED,0.85,1.50,1.00,4.84


In [31]:
shopper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   shopper_id     600 non-null    object 
 1   seniority      600 non-null    object 
 2   found_rate     600 non-null    float64
 3   picking_speed  600 non-null    float64
 4   accepted_rate  600 non-null    float64
 5   rating         600 non-null    float64
dtypes: float64(4), object(2)
memory usage: 28.2+ KB


#### Guardando dataset

In [32]:
#  La función pickle.dump() permite guardar el dataframe en un archivo en formato plk
pickle.dump(shopper, open('data/output/shopper.plk', 'wb'))

### Dataset: tienda (store)

In [33]:
# Cargando datos de tienda desde el archivo especificado
store = read_csv('data/input/4_tienda.csv', sep = ';', encoding='utf-8')
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(store.head(n = 5))

,store_branch_id,store,lat,lng
0,cf0895f1e3beb8164f0a86e6f940ae56,c0b93cf93cf854981b8d6bd849eec4b0,18.160.308.413.602.800,-10.026.067.265.111.100
1,3ddd4c29cab4178d89d549c6ba65c236,044d56734d20e04984d846dade6d39f3,1.827.282.968.914.450,-10.031.881.435.593.900
2,e9763f24fd46d0b9e52d423207a87c3c,452ae015bf846e1d2932b7ea15720d64,18.273.494.015.704.800,-10.031.958.539.978.200
3,49b7d60d6095a9f1129ce798102b62f4,6447951cd071f6f045fd4e6194920926,1.824.774.341.566.610,-10.040.355.771.938.600
4,8f21cfda17b340aeee062220d0bee8e5,452ae015bf846e1d2932b7ea15720d64,18.203.111.495.565.700,-10.029.771.361.550.100


In [34]:
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
store.columns.to_list()

['store_branch_id', 'store', 'lat', 'lng']

#### Normalizando nombre de columnas

In [35]:
# Asigna al dataframe el resultado de llamar a la función clean_columns().
# Esta función tiene como objetivo limpiar los nombres de las columnas del DataFrame.
# Elimina los caracteres especiales y los espacios los reemplaza por un _
store = clean_columns(store, case = 'snake')
# Obtiene una lista de nombres de columnas de un DataFrame de pandas.
store.columns.to_list()

0 column names have been cleaned

['store_branch_id', 'store', 'lat', 'lng']

#### Imputando valores perdidos

In [36]:
# Dataset no tiene valores perdidos

#### Imputando valores duplicados

In [37]:
store = store.drop_duplicates()
store = store.reset_index(drop=True)

#### Normalizando columnas para eliminar puntos adicionales y ajustando largo de decimales

In [38]:
# Eliminando . (puntos) despues del primer punto a la izquierda
# Rransformando las columnas "lat" y "lng" del DataFrame.
# La función DataFrame.map() aplica una función lambda a cada elemento de la columna seleccionada
# y devuelve una serie con los resultados.
# En este caso, la función lambda recibe como argumento x y reemplaza cada "." en el string con una cadena vacía
# (es decir, elimina los puntos decimales). Luego, 
# se extraen los primeros dos dígitos para la columna "lat" 
# y los primeros tres dígitos para la columna "lng",
# luego se concatenan con un nuevo punto decimal y los siguientes cinco dígitos del string original 
# (considerando la posición inicial dependiendo de la columna).
# Por último, se utiliza la función Series.astype() para cambiar el tipo de datos de las columnas "lat" y "lng" a float64.
# Esto se hace para que los valores de las columnas puedan ser tratados como números.
# store['lat'] = store['lat'].map(lambda x: x[0:x.find('.')] + '.' + x[x.find('.'):].replace('.', '')[0:5]).astype(float)
# store['lng'] = store['lng'].map(lambda x: x[0:x.find('.')] + '.' + x[x.find('.'):].replace('.', '')[0:5]).astype(float)
store['lat'] = store['lat'].map(lambda x: x.replace('.', '')[:2] + '.' + x.replace('.', '')[2:7]).astype('float64')
store['lng'] = store['lng'].map(lambda x: x.replace('.', '')[:3] + '.' + x.replace('.', '')[3:8]).astype('float64')

##### Inspeccion visual transformaciones

In [39]:
# Usamos la función option_context para establecer el número máximo de columnas para mostrar
with option_context("display.max_columns", None):
# Usamos la función de visualización para mostrar las primeras 5 filas de los datos
    display(store.head(n = 5))

,store_branch_id,store,lat,lng
0,cf0895f1e3beb8164f0a86e6f940ae56,c0b93cf93cf854981b8d6bd849eec4b0,18.16030,-10.02606
1,3ddd4c29cab4178d89d549c6ba65c236,044d56734d20e04984d846dade6d39f3,18.27282,-10.03188
2,e9763f24fd46d0b9e52d423207a87c3c,452ae015bf846e1d2932b7ea15720d64,18.27349,-10.03195
3,49b7d60d6095a9f1129ce798102b62f4,6447951cd071f6f045fd4e6194920926,18.24774,-10.04035
4,8f21cfda17b340aeee062220d0bee8e5,452ae015bf846e1d2932b7ea15720d64,18.20311,-10.02977


In [40]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   store_branch_id  113 non-null    object 
 1   store            113 non-null    object 
 2   lat              113 non-null    float64
 3   lng              113 non-null    float64
dtypes: float64(2), object(2)
memory usage: 3.7+ KB


#### Guardando dataset

In [41]:
#  La función pickle.dump() permite guardar el dataframe en un archivo en formato plk
pickle.dump(store, open('data/output/store.plk', 'wb'))